In [ ]:
import os
import cudf
from cuml.preprocessing import LabelEncoder

ROOT = "/rapids/notebooks/host/kaggle/kaggle"

# Load and preprocess data
# https://docs.dask.org/en/latest/remote-data-services.html
gdf_train = cudf.read_csv(os.path.join(ROOT, "train.csv"))
#gdf_train = cudf.read_csv("s3://jheaton-load-data/kaggle/train.csv", storage_options = {'anon': True})
#gdf_train = cudf.read_csv("s3://jheaton-load-data/kaggle/train.csv", storage_options = {'key': '', 'secret':''})
x = gdf_train.iloc[:,1:-1]
y = gdf_train.loc[:,'target']
y = LabelEncoder().fit_transform(y)

In [ ]:
%%time
import xgboost as xgb

# Train model
params = dict(
    objective='multi:softmax',
    max_depth=11,
    num_class=9,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.7,
    random_state=42,
    eval_metric='mlogloss',
    tree_method='gpu_hist',
    n_gpus = 1)
dtrain = xgb.DMatrix(x,y)
evallist = [(dtrain, 'train')]
clf = xgb.train(params, dtrain, 100, evallist)